In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.chains import RetrievalQA
from pathlib import Path
import gradio as gr
from langchain.tools import Tool
import os

In [8]:
class ProjectRootDetector:
    def __init__(self):
        self.root_indicators = [
            '.git',        
            'package.json',  
            'requirements.txt', 
            'pyproject.toml', 
            'Cargo.toml',     
            'go.mod',       
            'pom.xml',       
            'build.gradle',  
            'Dockerfile',  
            'docker-compose.yml',
            'README.md',
            'setup.py',
            'CMakeLists.txt'
        ]
    
    def find_root(self, use_file_path_as_root=False, root_path=None):
        """Find project root starting from any file path"""
        if root_path: 
            return root_path

        if use_file_path_as_root:
            return str(Path.cwd())
            
        current_path = Path.cwd()
        
        # Walk up the directory tree
        while current_path != current_path.parent:
            for indicator in self.root_indicators:
                if (current_path / indicator).exists():
                    return str(current_path)
            current_path = current_path.parent
       
        return str(Path.cwd())

In [ ]:
class RepoIntelligentAssistant:
    system_prompt = """
    You are an intelligent software engineer and codebase assistant with deep expertise in all programming domains (software engineering, data science, web development, etc.).

    BEHAVIOR GUIDELINES:

    1. QUESTION EVALUATION:
    - First determine if the question requires repository context or general coding knowledge
    - For repo-specific questions: Use tools to fetch relevant code/files
    - For general coding questions: Use your expertise to answer directly

    2. CODE EXPLANATION:
    - When explaining code: Break it down line-by-line
    - Show the actual code and explain what each line does
    - Focus on the specific implementation in the repository
    - Explain the purpose and context within the codebase

    3. CODE IMPROVEMENTS:
    - When suggesting improvements: Analyze the repository structure first
    - If suggesting new functions/classes: Write complete, working code
    - Base suggestions on the existing codebase patterns and architecture
    - Show the user the complete code implementation

    4. CODE MODIFICATION:
    - NEVER modify files directly
    - Always show the user the updated code
    - Let the user decide whether to implement changes
    - Provide clear instructions on where/how to apply changes

    5. RESPONSE SEQUENCE:
    a) Evaluate question type (repo-specific vs general)
    b) If repo-specific: Fetch relevant context using tools
    c) Provide detailed explanation with code examples
    d) If suggesting improvements: Show complete code implementation
    e) Always explain the reasoning behind suggestions

    Your tone: Precise, technical, and helpful - like a senior developer mentoring a colleague.
    """

    MODEL = "gpt-4o-mini"

    def __init__(self, use_file_path_as_root = False, root_path=None):
        self.llm = ChatOpenAI(
            temperature=0.7, 
            callbacks=[StreamingStdOutCallbackHandler()],
            model_name=self.MODEL
        )
        self.root_folder = ProjectRootDetector().find_root(root_path=root_path, use_file_path_as_root=use_file_path_as_root)
        self.agent = None  # Initialize agent as None

       
    def get_file(self, file_name: str):
        matches = []
        for root, dirs, files in os.walk(self.root_folder):
            print(files, flush=True)
            for f in files:
                if f == file_name:
                    matches.append(os.path.join(root, f))

        if not matches:
            return f"No file named '{file_name}' found in the repository."

        if len(matches) > 1:
            listed = "\n".join(matches)
            return (
                f"There are multiple files named '{file_name}'. "
                f"Please specify the full path.\n\n{listed}"
            )

        file_path = matches[0]
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()
            return f"Content of '{file_name}':\n\n{content}"
        except Exception as e:
            return f"Error reading file: {e}"
        
    
    def _get_all_folders_recursive(self, root_path):
        """Get all folders recursively from root path"""
        all_folders = []
        restricted_extensions = {
            '.exe', '.dll', '.so', '.dylib', '.bin', '.app', '.deb', '.rpm',
            '.zip', '.tar', '.gz', '.rar', '.7z', '.bz2', '.xz',
            '.pyc', '.pyo', '.pyd', '.class', '.jar', '.war',
            '.db', '.sqlite', '.sqlite3', '.mdb', '.accdb'
        }

        skip_folders = {
            '.git', '.svn', '.hg', '.bzr', 'node_modules', 'bower_components', 'vendor', '.venv',
            '__pycache__', '.pytest_cache', '.coverage', 'venv', '.venv', 'env', '.env',
            'dist', 'build', 'target', 'out', 'bin', 'obj', '.vscode', '.idea', '.vs', '.eclipse',
            '.DS_Store', 'Thumbs.db', '.Trash', 'temp', 'tmp', 'cache', 'logs', '.cache', '.tmp',
            'packages', 'lib', 'libs', 'dependencies', 'generated', 'auto-generated', '.generated',
            'backup', 'backups', '.backup', 'coverage', '.nyc_output', 'htmlcov','docs/_build', 'site', '_site'
        }
        
        for root, dirs, files in os.walk(root_path):
            dirs[:] = [d for d in dirs if d not in skip_folders]
            
            root_name = os.path.basename(root)
            if root_name in skip_folders:
                continue
                
            has_restricted = any(os.path.splitext(file)[1].lower() in restricted_extensions for file in files)
            if has_restricted:
                continue

            all_folders.append(root)
                
        return all_folders

    def create_doc(self):
        folders = self._get_all_folders_recursive(self.root_folder)
        documents = []
        for folder in folders:
            doc_type = os.path.basename(folder)
            print(doc_type)
            loader = DirectoryLoader(
                folder, 
                glob=["**/*.py", "**/*.ipynb", "**/*.md", "**/*.txt", "**/*.json", "**/*.csv", "**/*.html", "**/*.css", "**/*.js", "**/*.ts", "**/*.java", "**/*.c", "**/*.cpp", "**/*.go", "**/*.rust", "**/*.scala", "**/*.swift", "**/*.kotlin"], 
                loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"}
            )
            folder_docs = loader.load()
            for doc in folder_docs:
                doc.metadata["doc_type"] = doc_type
                documents.append(doc)
        return documents
    
    def create_retriever(self):
        embeddings =  OpenAIEmbeddings()

        documents = self.create_doc()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = text_splitter.split_documents(documents)

        vectorstore = FAISS.from_documents(chunks, embeddings)

        return vectorstore.as_retriever(k=30)

    def create_agent(self):
        retriever = self.create_retriever()
        rag_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            retriever=retriever,
            chain_type="stuff"
        )

        # Wrap RAG chain as a Tool
        rag_tool = Tool(
            name="repo_context_search",
            func=lambda q: rag_chain.run(q),
            description="Search the repository for specific code, functions, classes, or implementation details. Use this when the user asks about something specific in the codebase like 'what does this function do', 'explain this code', or 'how is X implemented'."
        )

        @tool("get_file", return_direct=False)
        def get_file(x):
            """
            Fetch the content of a specific file by name to analyze its code implementation.
            Use when user asks to 'explain this file', 'show me the code for X', or 'what's in this file'.
            If multiple files share the same name, list their paths instead.
            """
            return self.get_file(x)

        tools = [rag_tool, get_file]
        # Alternative: Use system message in memory
        memory = ConversationBufferMemory(
            memory_key="chat_history", 
            input_key="input", 
            return_messages=True,
            system_message=self.system_prompt  # Add system message here
        )

        agent = initialize_agent(
            tools,
            self.llm,
            agent_type=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
            handle_parsing_errors=True,
            memory=memory,
            verbose=True
            # Remove agent_kwargs since we're using memory system_message
        )

        return agent

    def run(self, question, history=None):
        try:
            # Create agent only once
            if not self.agent:
                self.agent = self.create_agent()
            
            response = self.agent.stream(question)
            for chunk in response:
                if chunk.get("output"):
                    yield chunk["output"]
        except Exception as e:
            yield f"Error: {str(e)}"

            

In [10]:
G = RepoIntelligentAssistant(root_path="C:/Users/hp/projects/gen-ai/llm_engineering/week5/community-contributions/tourist-guide/")

In [ ]:
interface = gr.ChatInterface(G.run, type="messages")
interface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


C:\Users\hp\AppData\Local\Temp\ipykernel_44004\2054011777.py:131: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings =  OpenAIEmbeddings()


C:\Users\hp\AppData\Local\Temp\ipykernel_44004\2054011777.py:167: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
C:\Users\hp\AppData\Local\Temp\ipykernel_44004\2054011777.py:174: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new None chain...
I need to search the repository for the specific implementation of the `load_knowledge_base` function to explain its functionality line by line.  
Action: repo_context_search  
Action Input: "load_knowledge_base"  

C:\Users\hp\AppData\Local\Temp\ipykernel_44004\2054011777.py:152: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  func=lambda q: rag_chain.run(q),


...

c:\Users\hp\projects\gen-ai\llm_engineering\.venv\Lib\site-packages\langchain\chains\base.py:487: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'messages']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  self.memory.save_context(inputs, outputs)




> Entering new None chain...
I need to search the repository for the `search_attractions` function to understand its implementation and then explain each line of the code. 

Action: repo_context_search  
Action Input: "search_attractions"  
Observation: I don't know.
Thought:It seems I couldn't locate the `search_attractions` function directly. I will try to search for it again with a more general query to see if I can find any relevant information about it. 

Action: repo_context_search  
Action Input: "def search_attractions"  
Observation: I'm sorry, but I don't know the answer to that.
Thought:It appears that I'm unable to find the `search_attractions` function through my current searches. To provide a detailed explanation, I will try to fetch specific files that may contain the function definition. 

Action: get_file  
Action Input: "search_attractions.py"  # Assuming this file may contain the function based on common naming conventions.  ['README.md', 'requirements.txt', 'style

c:\Users\hp\projects\gen-ai\llm_engineering\.venv\Lib\site-packages\langchain\chains\base.py:487: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'messages']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  self.memory.save_context(inputs, outputs)




> Entering new None chain...
I need to locate the `search_attractions` function within the `tourist-assistant` file to understand its implementation and explain each line of code. I'll start by searching for the specific function in the relevant file.

Action: repo_context_search  
Action Input: search_attractions function in tourist-assistant  

Observation: I don't know.
Thought:I need to fetch the content of the `tourist-assistant` file to find the `search_attractions` function directly and understand its implementation. 

Action: get_file  
Action Input: tourist-assistant  
['README.md', 'requirements.txt', 'style.css', 'tourist-assistant.py', 'travel.jpg']

Observation: No file named 'tourist-assistant  
' found in the repository.
Thought:It seems that there is no file named 'tourist-assistant' in the repository. I will search for any file that might contain the keyword 'search_attractions' to locate its implementation.

Action: repo_context_search  
Action Input: search_attract

c:\Users\hp\projects\gen-ai\llm_engineering\.venv\Lib\site-packages\langchain\chains\base.py:487: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'messages']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  self.memory.save_context(inputs, outputs)
